# Imports

In [4]:
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama
from langchain.evaluation import load_evaluator
import faiss
import pandas as pd
import numpy as np
import pickle
import time
from tqdm import tqdm

# Intialization

In [5]:
# Load the FAISS index
index = faiss.read_index("database/pdf_sections_index.faiss")

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
with open('database/pdf_sections_data.pkl', 'rb') as f:
        sections_data = pickle.load(f)

# RAG functions

In [8]:
def search_faiss(query, k=3):
    query_vector = model.encode([query])[0].astype('float32')
    query_vector = np.expand_dims(query_vector, axis=0)
    distances, indices = index.search(query_vector, k)
    
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        results.append({
            'distance': dist,
            'content': sections_data[idx]['content'],
            'metadata': sections_data[idx]['metadata']
        })
    
    return results

In [9]:
# Create a prompt template
prompt_template = """
You are an AI assistant specialized in dietary guidelines. 
Use the following pieces of context to answer the question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

llm = Ollama(
    model="llama3"
)

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

def answer_question(query):
    # Search for relevant context
    search_results = search_faiss(query)
    
    # Combine the content from the search results
    context = "\n\n".join([result['content'] for result in search_results])

    # Run the chain
    response = chain.run(context=context, question=query)
    
    return response

/var/folders/jd/5727mf8x4yb4knqc1wx3w4kw0000gn/T/ipykernel_14683/2553128167.py:16: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
/var/folders/jd/5727mf8x4yb4knqc1wx3w4kw0000gn/T/ipykernel_14683/2553128167.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


# Reading GT

In [10]:
df = pd.read_csv('data/QA_pairs.csv')

In [11]:
time_list=[]
response_list=[]
for i in tqdm(range(len(df))):
    query = df['Questions'].values[i]
    start = time.time()
    response = answer_question(query)
    end = time.time()   
    time_list.append(end-start)
    response_list.append(response)

  0%|          | 0/10 [00:00<?, ?it/s]/var/folders/jd/5727mf8x4yb4knqc1wx3w4kw0000gn/T/ipykernel_14683/2553128167.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(context=context, question=query)
100%|██████████| 10/10 [01:48<00:00, 10.83s/it]


In [12]:
df['latency'] = time_list
df['response'] = response_list

# Evaluation

In [13]:
eval_llm = Ollama(
    model="phi3"
)

In [14]:
metrics = ['correctness', 'relevance', 'coherence', 'conciseness']

In [16]:
for metric in metrics:
    evaluator = load_evaluator("labeled_criteria", criteria=metric, llm=eval_llm)
    
    reasoning = []
    value = []
    score = []
    
    for i in tqdm(range(len(df))):
        eval_result = evaluator.evaluate_strings(
            prediction=df.response.values[i],
            input=df.Questions.values[i],
            reference=df.Answers.values[i]
        )
        reasoning.append(eval_result['reasoning'])
        value.append(eval_result['value'])
        score.append(eval_result['score'])
    
    df[metric+'_reasoning'] = reasoning
    df[metric+'_value'] = value
    df[metric+'_score'] = score 

100%|██████████| 10/10 [02:36<00:00, 15.67s/it]


In [17]:
df[['correctness_score','relevance_score','coherence_score','conciseness_score','latency']].mean()

correctness_score     0.875000
relevance_score       0.888889
coherence_score       0.900000
conciseness_score     0.600000
latency              10.831184
dtype: float64

In [18]:
irr_q=pd.read_csv('data/irrelevant_questions.csv')

In [19]:
time_list=[]
response_list=[]
for i in tqdm(range(len(irr_q))):
    query = irr_q['Questions'].values[i]
    start = time.time()
    response = answer_question(query)
    end = time.time()   
    time_list.append(end-start)
    response_list.append(response)

100%|██████████| 10/10 [01:47<00:00, 10.73s/it]


In [20]:
irr_q['response']=response_list
irr_q['latency']=time_list

In [21]:
irr_q['irrelevant_score'] = irr_q['response'].str.contains("I don't know")

In [22]:
irr_q[['irrelevant_score','latency']].mean()

irrelevant_score     0.800000
latency             10.728183
dtype: float64

# Improvement

In [23]:
new_prompt_template = """
You are an AI assistant specialized in dietary guidelines. 
Use the following pieces of context to answer the question concisely. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=new_prompt_template, input_variables=["context", "question"])

llm = Ollama(
    model="llama3"
)

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

def answer_question_new(query):
    # Search for relevant context
    search_results = search_faiss(query)
    
    # Combine the content from the search results
    context = "\n\n".join([result['content'] for result in search_results])

    # Run the chain
    response = chain.run(context=context, question=query)
    
    return response

In [24]:
df2=df.copy()

In [25]:
time_list=[]
response_list=[]
for i in tqdm(range(len(df2))):
    query = df2['Questions'].values[i]
    start = time.time()
    response = answer_question(query)
    end = time.time()   
    time_list.append(end-start)
    response_list.append(response)

100%|██████████| 10/10 [02:18<00:00, 13.83s/it]


In [26]:
df2['latency'] = time_list
df2['response'] = response_list

In [27]:
for metric in metrics:
    evaluator = load_evaluator("labeled_criteria", criteria=metric, llm=eval_llm)
    
    reasoning = []
    value = []
    score = []
    
    for i in tqdm(range(len(df2))):
        eval_result = evaluator.evaluate_strings(
            prediction=df2.response.values[i],
            input=df2.Questions.values[i],
            reference=df2.Answers.values[i]
        )
        reasoning.append(eval_result['reasoning'])
        value.append(eval_result['value'])
        score.append(eval_result['score'])
    
    df2[metric+'_reasoning'] = reasoning
    df2[metric+'_value'] = value
    df2[metric+'_score'] = score 

100%|██████████| 10/10 [08:28<00:00, 50.88s/it]


In [28]:
df2[['correctness_score','relevance_score','coherence_score','conciseness_score','latency']].mean()

correctness_score     0.500000
relevance_score       0.000000
coherence_score       1.000000
conciseness_score     1.000000
latency              13.829776
dtype: float64

# Query relevance

In [29]:
def new_search_faiss(query, k=3, threshold=0.5):
    query_vector = model.encode([query])[0].astype('float32')
    query_vector = np.expand_dims(query_vector, axis=0)
    distances, indices = index.search(query_vector, k)
    
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if dist < threshold:  # Only include results within the threshold distance
            results.append({
                'distance': dist,
                'content': sections_data[idx]['content'],
                'metadata': sections_data[idx]['metadata']
            })
    
    return results

In [33]:
new_prompt_template = """
You are an AI assistant specialized in dietary guidelines. 
Use the following pieces of context to answer the question concisely. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

llm = Ollama(
    model="llama3"
)

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

def new_answer_question(query):
    # Search for relevant context
    search_results = new_search_faiss(query)
    
    if search_results==[]:
        response="I don't know"
    else:
        context = "\n\n".join([result['content'] for result in search_results])
        response = chain.run(context=context, question=query)
    
    return response

In [34]:
irr_q2=irr_q.copy()

In [35]:
time_list=[]
response_list=[]
for i in tqdm(range(len(irr_q2))):
    query = irr_q['Questions'].values[i]
    start = time.time()
    response = new_answer_question(query)
    end = time.time()   
    time_list.append(end-start)
    response_list.append(response)

100%|██████████| 10/10 [00:00<00:00, 28.13it/s]


In [36]:
irr_q2['response']=response_list
irr_q2['latency']=time_list

In [37]:
irr_q2['irrelevant_score'] = irr_q2['response'].str.contains("I don't know")

In [38]:
irr_q2[['irrelevant_score','latency']].mean()

irrelevant_score    1.000000
latency             0.035431
dtype: float64